# Mapping Embedding spaces

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from numpy.linalg import norm

### this functoin normalizes embedding vectors

In [2]:
from sklearn.preprocessing import normalize
def normalizer (df) :

  df = df.drop(columns=['sura', 'aya']) 
  df = df.drop(df.columns[[0]], axis=1) 
  normal = normalize(df.to_numpy(), axis=1, norm='l2')

  return torch.from_numpy(normal)

### for shuffling indexes in the case split data to train and validation

In [3]:
def shuffle_index(tensor1, tensor2) :

  res = torch.cat((tensor1, tensor2), 1)
  res = pd.DataFrame(res.numpy())
  return res.sample(frac = 1)  

### load embedding of different languages, normalize them and convert dataframes to Tensors

In [4]:
# reading embeddings
df = pd.read_csv('drive/MyDrive/Sources/Quran_en_vector.csv')
df1 = pd.read_csv('drive/MyDrive/Sources/arabian_embeddings.csv')
df2 = pd.read_csv('drive/MyDrive/Sources/persina_embeddings.csv')

print(df.shape)
print(df1.shape)
print(df2.shape)

########### normalize embedding vectors and convert them to tensors ############
en = normalizer(df)
ar = normalizer(df1)
fa = normalizer(df2)

df


####################### shuffle indexes for train/val###########################
#shuffled = shuffle_index(fa, en)
#shuffled

#train_idx = shuffled.iloc[:4000, :].index
#test_idx = shuffled.iloc[4000:, :].index

#X = torch.from_numpy(train_idx.iloc[:, :100].to_numpy())
#Y = torch.from_numpy(train_idx.iloc[:, 100:].to_numpy())
################################################################################


(6236, 103)
(6236, 103)
(6236, 103)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,92,93,94,95,96,97,98,99,sura,aya
0,0,0.130459,-0.170920,-0.130430,-0.425366,0.263938,-0.201712,-0.401716,-0.177249,-0.301538,...,0.043766,-0.082393,-0.055821,-0.447854,0.080386,-0.177847,-0.070858,0.376248,1,1
1,1,-0.137422,0.188611,-0.318364,-0.265647,0.076292,-0.156067,-0.147548,-0.022625,-0.183700,...,0.022016,-0.072181,0.030498,-0.300821,0.188728,-0.014946,-0.022813,0.142978,1,2
2,2,-0.105227,-0.367590,0.057834,-0.623360,0.127633,-0.038408,-0.425345,-0.320201,-0.604526,...,-0.340051,-0.145041,-0.165572,-0.362329,-0.119050,-0.301679,0.071205,0.355422,1,3
3,3,-0.126925,0.020166,0.100932,-0.464344,0.496763,-0.215234,-0.526151,0.088354,-0.222541,...,-0.196931,-0.343176,-0.005231,-0.042328,-0.142382,-0.003090,-0.035455,0.143022,1,4
4,4,-0.060498,0.123393,-0.069733,-0.197850,0.097257,-0.048006,-0.041843,-0.272861,-0.017689,...,0.123482,-0.000749,-0.020878,-0.352778,-0.044268,-0.035371,-0.078913,0.105660,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,6231,-0.077220,-0.161203,0.018463,-0.457005,0.416739,0.028342,-0.353683,0.014565,-0.536690,...,-0.317391,-0.395045,0.061353,-0.082348,-0.025392,-0.158036,0.239290,0.021167,114,2
6232,6232,-0.032591,0.205356,0.195381,-0.296000,0.302850,-0.017343,-0.317731,0.061563,-0.385725,...,-0.159178,-0.255210,0.135381,-0.090296,-0.139390,0.081853,0.071966,0.129449,114,3
6233,6233,-0.094378,0.350528,-0.063975,-0.493669,0.322647,-0.105747,-0.243063,0.020720,-0.429954,...,-0.072447,-0.226918,0.047581,-0.024180,-0.108588,0.060664,0.068632,0.147306,114,4
6234,6234,-0.057526,0.246342,-0.407510,-0.405858,0.120783,-0.312137,-0.258789,-0.029895,-0.563389,...,0.075562,-0.109152,0.039109,-0.225467,-0.039532,0.263450,-0.150685,0.219159,114,5


### Simple one layer Linear model

In [5]:
class LinearModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  
        
        #with torch.no_grad():
           # self.linear.weight.div_(torch.norm(self.linear.weight, dim=1, keepdim=True))

    def forward(self, x):
        #self.linear.weight = torch.nn.Parameter(self.linear.weight/torch.norm(self.linear.weight))
        #self.linear.bias = torch.nn.Parameter(self.linear.bias/torch.norm(self.linear.bias))
        out = self.linear(x)
        return out

### Train Loop

In [6]:
from torch.nn.functional import normalize
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable

input_dim = 100
output_dim = 100

model = LinearModel(input_dim, output_dim).cuda()

criterion = nn.MSELoss()
learning_rate = 1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.2)
scheduler = StepLR(optimizer, step_size=3000, gamma=0.9)


epochs = 50000
model.train()

for epoch in range(epochs):
    
    # Convert numpy array to torch Variable
    
    inputs = ar.cuda()
    labels = fa.cuda()

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad() 
    model.double()

    # Forward to get output
    #outputs = torch.from_numpy(normalize(model(inputs).detach().numpy(), axis=1, norm='l2')) 
    #model.linear.weight = torch.nn.Parameter(normalize(model.linear.weight))
    #model.linear.bias = torch.nn.Parameter(model.linear.bias/torch.norm(model.linear.bias))
    #print(model.linear.weight)
    #print(normalize(model.linear.weight))
    outputs = model(inputs)
    #break
    #outputs = normalize(model(inputs))

    #print(torch.norm(outputs))
    #print(np.linalg.norm(outputs[0, :]))
    #loss.requres_grad = True

    #A = outputs[0, :].detach().numpy()
    #B = labels[0, :].detach().numpy()
    #print("epoche : " + str(epoch) + "     " + str(abs(np.dot(A,B)/(norm(A)*norm(B)))))


    # Calculate Loss
    loss = criterion(outputs, labels)

    # Getting gradients w.r.t. parameters
    loss.backward()

    # Updating parameters
    optimizer.step()

    #model.linear.weight = torch.nn.Parameter(normalize(model.linear.weight))
    #model.linear.bias = torch.nn.Parameter(model.linear.bias/torch.norm(model.linear.bias))
 
    if epoch % 1000 ==0 :
      print('epoch {}, loss {}'.format(epoch, loss.item()))
    
    epoch += 1

epoch 0, loss 0.017512896102450378
epoch 1000, loss 0.005089529311951608
epoch 2000, loss 0.004745139250117934
epoch 3000, loss 0.004492095218820194
epoch 4000, loss 0.004303643233320378
epoch 5000, loss 0.004161548720518299
epoch 6000, loss 0.004053192886481015
epoch 7000, loss 0.003969713446667993
epoch 8000, loss 0.0039047979608895314
epoch 9000, loss 0.0038538898486982637
epoch 10000, loss 0.0038136586335936955
epoch 11000, loss 0.0037816415549483703
epoch 12000, loss 0.003755997789065981
epoch 13000, loss 0.0037353376520013145
epoch 14000, loss 0.0037186023901144495
epoch 15000, loss 0.0037049785442887406
epoch 16000, loss 0.0036938362418253356
epoch 17000, loss 0.0036846842496285866
epoch 18000, loss 0.003677136904974696
epoch 19000, loss 0.003670889555736944
epoch 20000, loss 0.0036657001602769085
epoch 21000, loss 0.0036613753894994714
epoch 22000, loss 0.0036577600495928952
epoch 23000, loss 0.00365472897490891
epoch 24000, loss 0.003652180772938138
epoch 25000, loss 0.0036500

In [7]:
torch.save(model, '/content/drive/MyDrive/Sources/ArToFa.pt')

In [32]:
ArToFa = torch.load( '/content/drive/MyDrive/Sources/ArToFa.pt').cuda()
ArToEn = torch.load( '/content/drive/MyDrive/Sources/ArToEn.pt').cuda()
FaToEn = torch.load( '/content/drive/MyDrive/Sources/FaToEn.pt').cuda()

### Translate Aya

In [25]:
def read_lines(filename) :
  with open('/content/drive/MyDrive/Sources/' + filename) as f:
    lines = f.readlines()

  sura, aya, text = [], [], []
  for line in lines:
      temp = line.split('|')
      if len(temp)==3:
        sura.append(int(temp[0]))
        aya.append(int(temp[1]))
        text.append(temp[2])
  df = pd.DataFrame()
  df['sura'] = sura
  df['aya'] = aya
  df['text'] = text

  return df

In [54]:
from sklearn.preprocessing import normalize

def translate_aya(index_aya, model, src, dst) : 

 # print(src[index_aya, :].shape)
 # print(dst.shape)

  src = src.cpu()
  dst = dst.cpu()
  model.cpu()
  A = normalize(model(src).detach().numpy(), axis=1, norm='l2')[index_aya, :]

  #print(A.shape)
  outs = []
  for i in range(6236) : 
  
    #A = model(src[index_aya, :]).detach().numpy()
    B = dst[i, :]
    outs.append(np.dot(A,B)/(norm(A)*norm(B)))

  print(max(outs))
  return outs.index(max(outs))

In [52]:
en_quran = read_lines('en.ahmedali.txt')
fa_quran = read_lines('fa.ansarian.txt')
ar_quran = read_lines('ar.jalalayn.txt')

fa_quran

,sura,aya,text
0,1,1,به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش...
1,1,2,همه ستایش ها، ویژه خدا، مالک و مربّی جهانیان ا...
2,1,3,رحمتش بی اندازه و مهربانی اش همیشگی است.\n
3,1,4,مالک و فرمانروای روز پاداش و کیفر است.\n
4,1,5,[پروردگارا!] تنها تو را می پرستیم وتنها از تو ...
...,...,...,...
6231,114,2,[به] پادشاه مردم\n
6232,114,3,[به] معبود مردم\n
6233,114,4,از زیان وسوسه گر کمین گرفته و پنهان،\n
6234,114,5,آنکه همواره در سینه های مردم وسوسه می کند\n


In [56]:
indexToTranslate = 1000

translated_index = translate_aya(indexToTranslate, FaToEn, fa, en)

print(fa_quran.iloc[indexToTranslate, :])

print(en_quran.iloc[translated_index, :])
#translated = en_quran[translate_aya(6231, FaToEn, fa, en), :]

0.9383789749361351
sura                                                    7
aya                                                    47
text    و چون بهشتیان، چشمانشان ناخواسته به سوی دوزخیا...
Name: 1000, dtype: object
sura                                                    9
aya                                                    74
text    They swear by God: "We never said this." But t...
Name: 1308, dtype: object
